### #5

In [26]:
import tensorflow as tf
import time
import os
import numpy as np
import cv2
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [27]:
y = []
img_vectors = []
folder_path = 'GTSRB'
width = 32
height = 32
for idx, folder_name in enumerate(os.listdir(folder_path)):
    for filename in (os.listdir(os.path.join(folder_path, folder_name))):
        img = cv2.imread(os.path.join(folder_path, folder_name, filename))
        img = cv2.resize(img, (width, height))
        img_vectors.append(img)
        y.append(idx)

In [28]:
len(y)

8910

In [29]:
y = np.asarray(y)

In [30]:
y.shape

(8910,)

In [31]:
len(img_vectors)

8910

In [32]:
X = np.asarray(img_vectors)

In [33]:
X.shape

(8910, 32, 32, 3)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [35]:
X_train.shape

(7128, 32, 32, 3)

In [36]:
y_train.shape

(7128,)

In [37]:
X_test.shape

(1782, 32, 32, 3)

In [38]:
y_test.shape

(1782,)

In [39]:
y_train = np_utils.to_categorical(y_train)

In [40]:
y_train.shape

(7128, 9)

In [41]:
layers = [tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(32, 32, 3)), 
         tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'), 
         tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'), 
         tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'), 
         tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'), 
         tf.keras.layers.Flatten(), 
         tf.keras.layers.Dense(units=128, activation='relu'), 
         tf.keras.layers.Dropout(0.5), 
         tf.keras.layers.Dense(units=9, activation='softmax')]
model = tf.keras.Sequential(layers)

In [42]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
train_start_time = time.time()
model.fit(x=X_train, y=y_train, epochs=10)
train_elapsed_time = time.time() - train_start_time

print('Train time is:', train_elapsed_time)

Epoch 1/10
223/223 [==============================] - 16s 73ms/step - loss: 2.5208 - accuracy: 0.3541
Epoch 2/10
223/223 [==============================] - 16s 72ms/step - loss: 0.7712 - accuracy: 0.7567
Epoch 3/10
223/223 [==============================] - 16s 73ms/step - loss: 0.3891 - accuracy: 0.8844
Epoch 4/10
223/223 [==============================] - 16s 72ms/step - loss: 0.2758 - accuracy: 0.9157
Epoch 5/10
223/223 [==============================] - 16s 73ms/step - loss: 0.2079 - accuracy: 0.9376
Epoch 6/10
223/223 [==============================] - 16s 73ms/step - loss: 0.1639 - accuracy: 0.9501
Epoch 7/10
223/223 [==============================] - 16s 72ms/step - loss: 0.1330 - accuracy: 0.9603
Epoch 8/10
223/223 [==============================] - 16s 72ms/step - loss: 0.1496 - accuracy: 0.9582
Epoch 9/10
223/223 [==============================] - 16s 74ms/step - loss: 0.0979 - accuracy: 0.9711
Epoch 10/10
223/223 [==============================] - 16s 73ms/step - loss: 0.067

In [44]:
prediction_start_time = time.time()
y_pred = model.predict(X_test)
prediction_elapsed_time = time.time() - prediction_start_time

print('Prediction time is:', prediction_elapsed_time)

Prediction time is: 1.1458101272583008


In [45]:
y_pred.shape

(1782, 9)

In [46]:
y_pred = np.argmax(y_pred, axis=1)

In [47]:
y_pred

array([5, 8, 1, ..., 6, 2, 1], dtype=int64)

In [48]:
y_test

array([5, 8, 1, ..., 6, 2, 1])

In [49]:
print('Prediction accuracy: {:.2f} %'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy: 97.87 %


In [50]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              